# Coursework 1

This notebook is intended to be used as a starting point for your experiments. The instructions can be found in the instructions file located under spec/coursework1.pdf. The methods provided here are just helper functions. If you want more complex graphs such as side by side comparisons of different experiments you should learn more about matplotlib and implement them. Before each experiment remember to re-initialize neural network weights and reset the data providers so you get a properly initialized experiment. For each experiment try to keep most hyperparameters the same except the one under investigation so you can understand what the effects of each are.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mlp.optimisers import Optimiser
%matplotlib inline
plt.style.use('ggplot')

def train_model_and_plot_stats(
        model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for num_epochs epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    ax_1.set_ylabel('Error')

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    ax_2.set_xlabel('Accuracy')
    
    return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

In [ ]:
import numpy as np
import logging
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider
from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import AdamLearningRule
from mlp.optimisers import Optimiser
def train_one_hidden_layer(
   model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True):
   
   # As well as monitoring the error over training also monitor classification
   # accuracy i.e. proportion of most-probable predicted classes being equal to targets
   data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

   # Use the created objects to initialise a new Optimiser instance.
   optimiser = Optimiser(
      model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)

   # Run the optimiser for num_epochs epochs (full passes through the training set)
   # printing statistics every epoch.
   stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)
   return stats, keys, run_time

def plot_one_hidden_layer(stats_list, keys_list, stats_interval, hidden_num_list):
   # Plot the change in the validation and training set error over training.
   fig_1 = plt.figure(figsize=(8, 4))
   ax_1 = fig_1.add_subplot(111)
   for index, hidden_num in enumerate(hidden_num_list):
      for k in ['width {}(train)'.format(hidden_num), 'width {}(valid)'.format(hidden_num)]:
         ax_1.plot(np.arange(1, stats_list[index].shape[0]) * stats_interval, 
                  stats_list[index][1:, keys_list[index][k]], label=k)
   ax_1.legend(loc=0)
   ax_1.set_xlabel('Epoch number')
   ax_1.set_ylabel('Error')

   # Plot the change in the validation and training set accuracy over training.
   fig_2 = plt.figure(figsize=(8, 4))
   ax_2 = fig_2.add_subplot(111)
   for index, hidden_num in enumerate(hidden_num_list):
      for k in ['width {}(train)'.format(hidden_num), 'width {}(valid)'.format(hidden_num)]:
         ax_2.plot(np.arange(1, stats_list[index].shape[0]) * stats_interval, 
                  stats_list[index][1:, keys_list[index][k]], label=k)
   ax_2.legend(loc=0)
   ax_2.set_xlabel('Epoch number')
   ax_2.set_xlabel('Accuracy')
   return fig_1, ax_1, fig_2, ax_2

def set_state(seed):
   # Seed a random number generator
   seed = 11102019 
   rng = np.random.RandomState(seed)
   return rng
   
def data_load(batch_size, rng):
   # Create data provider objects for the MNIST data set
   train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
   valid_data = EMNISTDataProvider('valid', batch_size=batch_size, rng=rng)
   return train_data, valid_data
   
def train_and_plot_one_hidden_layer(params):
   # set random state
   rng = set_state(params['seed'])
   # Set up a logger object to print info about the training run to stdout
   logger = logging.getLogger()
   logger.setLevel(logging.INFO)
   logger.handlers = [logging.StreamHandler()]
   # load data
   train_data, valid_data = data_load(params['batch_size'], rng)

   # data_stats and keys
   stats_list = []
   keys_list = []
   for hidden_dim in params['hidden_dim_list']:
      # initialize weights
      weights_init = GlorotUniformInit(rng=rng)
      biases_init = ConstantInit(0.)

      # Create model with ONE hidden layer
      model = MultipleLayerModel([
         AffineLayer(params['input_dim'], hidden_dim, weights_init, biases_init), # hidden layer
         ReluLayer(),
         AffineLayer(hidden_dim, params['output_dim'], weights_init, biases_init) # output layer
      ])
          
      error = CrossEntropySoftmaxError()
      # Use a Adam learning rule
      learning_rule = AdamLearningRule(learning_rate=params['learning_rate'])
      stats, keys, run_time = train_one_hidden_layer(model, error, learning_rule, train_data, valid_data, params['num_epochs'], params['stats_interval'], notebook=True)
      stats_list.append(stats)
      keys_list.append(keys)

      # dont forget to reset dataset!!!
      train_data.reset()
      valid_data.reset()
   # plot
   fig_1, ax_1, fig_2, ax_2 = plot_one_hidden_layer(stats_list, keys_list, params['stats_interval'], hidden_num_list = params['hidden_dim_list'])
   

In [ ]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.
import numpy as np
import logging
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider

# Seed a random number generator
seed = 11102019 
rng = np.random.RandomState(seed)
batch_size = 100
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
valid_data = EMNISTDataProvider('valid', batch_size=batch_size, rng=rng)

In [ ]:
# The model set up code below is provided as a starting point.
# You will probably want to add further code cells for the
# different experiments you run.

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import AdamLearningRule
from mlp.optimisers import Optimiser

# Setup hyperparameters
params = dict(
batch_size = 100,
learning_rate = 9e-4,
num_epochs = 100,
stats_interval = 1,
input_dim = 784, 
output_dim = 47,
hidden_dim_list = [32, 64, 128],
seed = 11102019)

# initialize the dataProvider and weights
train_data.reset()
valid_data.reset()
weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

# Create model with ONE hidden layer
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

error = CrossEntropySoftmaxError()
# Use a Adam learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate)

# Remember to use notebook=False when you write a script to be run in a terminal
_ = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)

In [ ]:
# Create model with TWO hidden layers
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # first hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), # second hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

error = CrossEntropySoftmaxError()
# Use a Adam learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate)

# Remember to use notebook=False when you write a script to be run in a terminal
_ = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)